## Uavs Collision

Collision of 2 uavs


In [1]:
import os
import numpy as np

# Custom modules
from helpers.change_coordinates import global2local
from helpers.visualization.plots import plot_3d_interactive
from plan import Plan,State
from gcs import GCS
from simulators.QGroundControl.qgc import QGC
from simulators.gazebo.gazebo import Gazebo


#Kill all related process
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

## Create Plans

In [2]:
offsets = [(-20, 0, 0, 0),(20, 0, 0, 0)]#
n_vehicles=len(offsets)


homes=np.array([offset[:3] for offset in offsets])
global_paths=[np.array([[-20, 0, 5],[20, 0, 5]]),
              np.array([[20, 0, 5],[-20, 0, 5]])]
local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5,navegation_speed=10) for path in local_paths]

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Set Navigation Speed' created — no connection yet 🧩
Step 'Set speed to 10.00 m/s' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (40, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm

## Visualization Parameters

In [3]:
waypoints = {f'waypoint_{i+1}': {'pos':global_paths[0],'color':'blue'} for i in range(n_vehicles)}
plot_3d_interactive(waypoints,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)

## Choose Simulator

In [ ]:
simulator=QGC(offsets=offsets,
              plans=plans,
              origin=(-35.3633245,149.1652241,0,0))

simulator=Gazebo(offsets=offsets,
                 plans=plans,
                 world_path='simulators/gazebo/worlds/runway.world',
                 vehicle_models=n_vehicles*['drone'],
                 markers=waypoints)

Launch Simulator

In [5]:
uavs = simulator.launch()

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'Set speed to 10.00 m/s' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 0, 5)' is now connected ✅🔗
Vehicle 1:

## Execute Plan

In [6]:
gcs=GCS(positions=homes.copy())
while gcs.incomplete_missions:
    to_remove = set()
    for id in gcs.incomplete_missions:
        uav = uavs[id-1]
        if uav.plan.state == State.DONE:
            to_remove.add(id)
        else:
            uav.plan.act()
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: Set speed to 10.00 m/s
Vehicle 1: ✅ Step: Set speed to 10.00 m/s is done
Vehicle 1: ▶️ Starting Step: arm
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: Set speed to 10.

In [7]:
plans[0]

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 10.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to  -> (40, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

In [8]:
plans[1]

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 10.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to  -> (-40, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

## Kill all related process

In [9]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")